# 📊 Análisis Exploratorio de Datos (EDA)
# Sistema de Evaluación de Riesgo Crediticio

Este notebook presenta un análisis exhaustivo de los datos utilizados para el sistema de recomendación de evaluación de riesgo crediticio.

## Objetivos del Análisis
1. **Exploración inicial**: Comprender la estructura y características de los datos
2. **Calidad de datos**: Identificar valores faltantes, outliers y inconsistencias
3. **Distribuciones**: Analizar la distribución de variables clave
4. **Correlaciones**: Identificar relaciones entre variables
5. **Insights crediticios**: Extraer patrones relevantes para evaluación de riesgo

---

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
import sys
import os

# Configurar visualizaciones
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configurar pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Librerías importadas correctamente")

## 1. 📂 Carga de Datos

In [ ]:
# Agregar directorio raíz al path
project_root = Path.cwd()
if project_root.name == 'notebooks':
    project_root = project_root.parent

sys.path.append(str(project_root))

# Importar módulos del proyecto
from src.data.load_data import DataLoader

# Inicializar cargador de datos
loader = DataLoader()

print(f"📁 Directorio del proyecto: {project_root}")
print(f"📁 Directorio de datos: {loader.processed_data_dir}")

In [ ]:
# Verificar si existen datos procesados, si no generarlos
data_file = loader.processed_data_dir / 'credit_data_processed.csv'

if not data_file.exists():
    print("🔄 Generando datos de muestra...")
    
    # Crear directorio si no existe
    loader.processed_data_dir.mkdir(parents=True, exist_ok=True)
    
    # Generar datos sintéticos
    df_raw = loader.create_sample_data(n_samples=2000)
    df_clean = loader.clean_data(df_raw)
    loader.save_processed_data(df_clean, 'credit_data_processed.csv')
    
    print("✅ Datos generados y guardados")
else:
    print("📊 Datos existentes encontrados")

# Cargar datos
df = pd.read_csv(data_file)
print(f"\n📈 Datos cargados: {df.shape[0]} filas, {df.shape[1]} columnas")

## 2. 🔍 Exploración Inicial

In [ ]:
# Información general del dataset
print("📋 INFORMACIÓN GENERAL DEL DATASET")
print("=" * 50)
print(f"Dimensiones: {df.shape[0]} filas × {df.shape[1]} columnas")
print(f"Memoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"\nTipos de datos:")
print(df.dtypes.value_counts())

In [ ]:
# Primeras filas del dataset
print("🔸 PRIMERAS 5 FILAS DEL DATASET")
print("=" * 50)
display(df.head())

In [ ]:
# Información detallada
print("📊 INFORMACIÓN DETALLADA")
print("=" * 50)
df.info()

## 3. 📈 Estadísticas Descriptivas

In [ ]:
# Estadísticas descriptivas para variables numéricas
print("📊 ESTADÍSTICAS DESCRIPTIVAS - VARIABLES NUMÉRICAS")
print("=" * 70)

numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if 'customer_id' in numerical_cols:
    numerical_cols.remove('customer_id')

display(df[numerical_cols].describe())

In [ ]:
# Estadísticas para variables categóricas
print("📊 ESTADÍSTICAS DESCRIPTIVAS - VARIABLES CATEGÓRICAS")
print("=" * 70)

categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

for col in categorical_cols:
    print(f"\n🔹 {col.upper()}:")
    print(f"Valores únicos: {df[col].nunique()}")
    print("Distribución:")
    print(df[col].value_counts())
    print("-" * 40)

## 4. 🎯 Análisis de la Variable Objetivo

In [ ]:
# Análisis de la variable objetivo (default)
print("🎯 ANÁLISIS DE LA VARIABLE OBJETIVO (DEFAULT)")
print("=" * 50)

# Distribución de la variable objetivo
default_counts = df['default'].value_counts()
default_props = df['default'].value_counts(normalize=True)

print(f"Distribución absoluta:")
print(f"  No Default (0): {default_counts[0]:,} ({default_props[0]:.1%})")
print(f"  Default (1): {default_counts[1]:,} ({default_props[1]:.1%})")
print(f"\nTasa de default: {default_props[1]:.2%}")

# Crear visualización
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Gráfico de barras
default_counts.plot(kind='bar', ax=axes[0], color=['skyblue', 'coral'])
axes[0].set_title('Distribución de Default (Conteos)')
axes[0].set_xlabel('Default')
axes[0].set_ylabel('Cantidad')
axes[0].set_xticklabels(['No Default', 'Default'], rotation=0)

# Gráfico circular
axes[1].pie(default_counts.values, labels=['No Default', 'Default'], 
           autopct='%1.1f%%', colors=['skyblue', 'coral'])
axes[1].set_title('Proporción de Default')

plt.tight_layout()
plt.show()

## 5. 📊 Distribución de Variables Numéricas

In [ ]:
# Distribución de variables numéricas principales
key_numerical = ['age', 'income', 'credit_score', 'debt_ratio', 'employment_years', 'loan_amount']
key_numerical = [col for col in key_numerical if col in df.columns]

# Crear subplots
n_cols = 3
n_rows = (len(key_numerical) + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes

for i, col in enumerate(key_numerical):
    # Histograma con separación por default
    df[df['default'] == 0][col].hist(alpha=0.7, bins=30, label='No Default', ax=axes[i])
    df[df['default'] == 1][col].hist(alpha=0.7, bins=30, label='Default', ax=axes[i])
    
    axes[i].set_title(f'Distribución de {col}')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frecuencia')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

# Ocultar subplots vacíos
for j in range(i+1, len(axes)):
    axes[j].set_visible(False)

plt.suptitle('Distribuciones de Variables Numéricas por Estado de Default', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Boxplots para identificar outliers
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes

for i, col in enumerate(key_numerical):
    # Boxplot separado por default
    data_to_plot = [df[df['default'] == 0][col], df[df['default'] == 1][col]]
    bp = axes[i].boxplot(data_to_plot, labels=['No Default', 'Default'], patch_artist=True)
    
    # Colorear las cajas
    bp['boxes'][0].set_facecolor('skyblue')
    bp['boxes'][1].set_facecolor('coral')
    
    axes[i].set_title(f'Boxplot de {col}')
    axes[i].set_ylabel(col)
    axes[i].grid(True, alpha=0.3)

# Ocultar subplots vacíos
for j in range(i+1, len(axes)):
    axes[j].set_visible(False)

plt.suptitle('Boxplots de Variables Numéricas por Estado de Default', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

## 6. 🔗 Análisis de Correlaciones

In [ ]:
# Matriz de correlación
correlation_matrix = df[numerical_cols].corr()

# Crear heatmap de correlación
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .5})
plt.title('Matriz de Correlación - Variables Numéricas')
plt.tight_layout()
plt.show()

# Correlación con la variable objetivo
print("\n🎯 CORRELACIÓN CON VARIABLE OBJETIVO (DEFAULT)")
print("=" * 50)
default_corr = df[numerical_cols].corrwith(df['default']).sort_values(key=abs, ascending=False)
print(default_corr)

In [ ]:
# Visualizar correlación con default
plt.figure(figsize=(10, 6))
default_corr_abs = default_corr.abs().sort_values(ascending=True)
colors = ['red' if x < 0 else 'green' for x in default_corr[default_corr_abs.index]]

plt.barh(range(len(default_corr_abs)), default_corr[default_corr_abs.index].values, 
         color=colors, alpha=0.7)
plt.yticks(range(len(default_corr_abs)), default_corr_abs.index)
plt.xlabel('Correlación con Default')
plt.title('Correlación de Variables Numéricas con Default')
plt.axvline(x=0, color='black', linestyle='-', alpha=0.3)
plt.grid(True, alpha=0.3)

# Agregar valores en las barras
for i, v in enumerate(default_corr[default_corr_abs.index].values):
    plt.text(v + 0.01 if v >= 0 else v - 0.01, i, f'{v:.3f}', 
             va='center', ha='left' if v >= 0 else 'right')

plt.tight_layout()
plt.show()

## 7. 📊 Análisis de Variables Categóricas

In [ ]:
# Análisis de variables categóricas vs default
categorical_analysis = ['employment_status', 'education_level', 'marital_status', 'housing_status']
categorical_analysis = [col for col in categorical_analysis if col in df.columns]

n_cols = 2
n_rows = (len(categorical_analysis) + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))

if n_rows == 1:
    axes = [axes] if n_cols == 1 else axes
else:
    axes = axes.flatten()

for i, col in enumerate(categorical_analysis):
    # Calcular tasas de default por categoría
    default_rates = df.groupby(col)['default'].agg(['count', 'sum', 'mean'])
    default_rates.columns = ['Total', 'Defaults', 'Default_Rate']
    default_rates = default_rates.sort_values('Default_Rate')
    
    # Crear gráfico de barras
    bars = axes[i].bar(range(len(default_rates)), default_rates['Default_Rate'], 
                      alpha=0.7, color='coral')
    axes[i].set_title(f'Tasa de Default por {col}')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Tasa de Default')
    axes[i].set_xticks(range(len(default_rates)))
    axes[i].set_xticklabels(default_rates.index, rotation=45, ha='right')
    axes[i].grid(True, alpha=0.3)
    
    # Agregar valores en las barras
    for bar, rate in zip(bars, default_rates['Default_Rate']):
        axes[i].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
                    f'{rate:.1%}', ha='center', va='bottom', fontsize=9)

# Ocultar subplots vacíos
for j in range(i+1, len(axes)):
    axes[j].set_visible(False)

plt.suptitle('Análisis de Variables Categóricas vs Default', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

## 8. 🔍 Detección de Valores Faltantes y Outliers

In [ ]:
# Análisis de valores faltantes
print("🔍 ANÁLISIS DE VALORES FALTANTES")
print("=" * 50)

missing_data = df.isnull().sum()
missing_percent = 100 * missing_data / len(df)

missing_table = pd.DataFrame({
    'Valores Faltantes': missing_data,
    'Porcentaje': missing_percent
})

missing_table = missing_table[missing_table['Valores Faltantes'] > 0].sort_values('Valores Faltantes', ascending=False)

if len(missing_table) > 0:
    print(missing_table)
    
    # Visualizar valores faltantes
    plt.figure(figsize=(10, 6))
    sns.barplot(x=missing_table['Porcentaje'], y=missing_table.index, palette='viridis')
    plt.title('Porcentaje de Valores Faltantes por Variable')
    plt.xlabel('Porcentaje de Valores Faltantes')
    plt.tight_layout()
    plt.show()
else:
    print("✅ No se encontraron valores faltantes en el dataset")

In [ ]:
# Detección de outliers usando IQR
print("🔍 DETECCIÓN DE OUTLIERS (MÉTODO IQR)")
print("=" * 50)

outlier_summary = []

for col in numerical_cols:
    if col == 'default':  # Saltar variable objetivo
        continue
        
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outlier_count = len(outliers)
    outlier_percent = 100 * outlier_count / len(df)
    
    outlier_summary.append({
        'Variable': col,
        'Outliers': outlier_count,
        'Porcentaje': outlier_percent,
        'Límite Inferior': lower_bound,
        'Límite Superior': upper_bound
    })

outlier_df = pd.DataFrame(outlier_summary)
outlier_df = outlier_df.sort_values('Porcentaje', ascending=False)

print(outlier_df.to_string(index=False))

## 9. 💡 Análisis Específico para Riesgo Crediticio

In [ ]:
# Análisis de ratios financieros importantes
print("💡 ANÁLISIS DE RATIOS FINANCIEROS")
print("=" * 50)

# Calcular ratios adicionales
df_analysis = df.copy()
df_analysis['loan_to_income'] = df_analysis['loan_amount'] / df_analysis['income']
df_analysis['monthly_income'] = df_analysis['income'] / 12

# Análisis por segmentos de riesgo
# Definir segmentos de puntaje crediticio
df_analysis['credit_segment'] = pd.cut(df_analysis['credit_score'], 
                                      bins=[0, 580, 670, 740, 800, 850],
                                      labels=['Poor', 'Fair', 'Good', 'Very Good', 'Excellent'])

# Definir segmentos de ingresos
df_analysis['income_segment'] = pd.qcut(df_analysis['income'], 
                                       q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

# Análisis cruzado: puntaje crediticio vs ingresos
cross_analysis = pd.crosstab(df_analysis['credit_segment'], 
                            df_analysis['income_segment'], 
                            df_analysis['default'], 
                            aggfunc='mean')

print("\n📊 Tasa de Default por Segmento de Crédito e Ingresos:")
print(cross_analysis.round(3))

In [ ]:
# Visualización del análisis cruzado
plt.figure(figsize=(10, 6))
sns.heatmap(cross_analysis, annot=True, fmt='.3f', cmap='Reds', 
           cbar_kws={'label': 'Tasa de Default'})
plt.title('Tasa de Default por Segmento de Crédito e Ingresos')
plt.xlabel('Segmento de Ingresos (Cuartiles)')
plt.ylabel('Segmento de Puntaje Crediticio')
plt.tight_layout()
plt.show()

In [ ]:
# Análisis de tendencias por edad y experiencia laboral
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Tendencia por edad
age_bins = pd.cut(df['age'], bins=range(18, 81, 5))
age_default = df.groupby(age_bins)['default'].mean()
age_default.plot(kind='line', ax=axes[0], marker='o', color='coral')
axes[0].set_title('Tasa de Default por Grupos de Edad')
axes[0].set_xlabel('Edad')
axes[0].set_ylabel('Tasa de Default')
axes[0].grid(True, alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

# Tendencia por experiencia laboral
emp_bins = pd.cut(df['employment_years'], bins=range(0, 41, 3))
emp_default = df.groupby(emp_bins)['default'].mean()
emp_default.plot(kind='line', ax=axes[1], marker='s', color='skyblue')
axes[1].set_title('Tasa de Default por Años de Empleo')
axes[1].set_xlabel('Años de Empleo')
axes[1].set_ylabel('Tasa de Default')
axes[1].grid(True, alpha=0.3)
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 10. 📋 Conclusiones y Recomendaciones

In [ ]:
# Resumen ejecutivo del análisis
print("📋 RESUMEN EJECUTIVO - ANÁLISIS EXPLORATORIO")
print("=" * 60)

# Estadísticas generales
total_clients = len(df)
default_rate = df['default'].mean()
avg_credit_score = df['credit_score'].mean()
avg_income = df['income'].mean()
avg_loan_amount = df['loan_amount'].mean()

print(f"\n📊 MÉTRICAS CLAVE:")
print(f"   • Total de clientes analizados: {total_clients:,}")
print(f"   • Tasa de default general: {default_rate:.2%}")
print(f"   • Puntaje crediticio promedio: {avg_credit_score:.0f}")
print(f"   • Ingreso promedio: ${avg_income:,.0f}")
print(f"   • Monto de préstamo promedio: ${avg_loan_amount:,.0f}")

# Variables más correlacionadas con default
top_correlations = df[numerical_cols].corrwith(df['default']).abs().sort_values(ascending=False)
print(f"\n🎯 VARIABLES MÁS PREDICTIVAS:")
for var, corr in top_correlations.head(5).items():
    if var != 'default':
        direction = "(+)" if df[var].corr(df['default']) > 0 else "(-)"
        print(f"   • {var}: {corr:.3f} {direction}")

# Segmentos de mayor riesgo
high_risk_employment = df.groupby('employment_status')['default'].mean().idxmax()
high_risk_education = df.groupby('education_level')['default'].mean().idxmax()

print(f"\n⚠️  SEGMENTOS DE ALTO RIESGO:")
print(f"   • Estado laboral: {high_risk_employment}")
print(f"   • Nivel educativo: {high_risk_education}")
print(f"   • Puntaje crediticio < 580: {(df[df['credit_score'] < 580]['default'].mean():.1%)} default rate")
print(f"   • Ratio deuda > 0.8: {(df[df['debt_ratio'] > 0.8]['default'].mean():.1%)} default rate")

In [ ]:
# Recomendaciones para el modelo de ML
print("\n💡 RECOMENDACIONES PARA MODELADO:")
print("=" * 50)

recommendations = [
    "1. 🎯 VARIABLES CLAVE: Priorizar credit_score, debt_ratio e income en el modelo",
    "2. ⚖️  BALANCEO: Considerar técnicas de balanceo para la clase minoritaria (default)",
    "3. 🔧 FEATURE ENGINEERING: Crear ratios como loan_to_income, payment_to_income",
    "4. 📊 SEGMENTACIÓN: Incluir variables categóricas codificadas (employment_status, education)",
    "5. 🔍 OUTLIERS: Aplicar técnicas robustas debido a la presencia de valores atípicos",
    "6. ✅ VALIDACIÓN: Usar validación cruzada estratificada para mantener proporción de clases",
    "7. 📈 MÉTRICAS: Enfocarse en Precision, Recall y AUC-ROC para evaluación del modelo",
    "8. 🎲 ENSEMBLE: Considerar modelos ensemble (Random Forest, XGBoost) para mejor rendimiento"
]

for rec in recommendations:
    print(f"   {rec}")

print("\n" + "=" * 60)
print("✅ ANÁLISIS EXPLORATORIO COMPLETADO")
print("Los datos están listos para el proceso de modelado")
print("=" * 60)